NN using word embedding
https://realpython.com/python-keras-text-classification/

In [1]:
import sqlite3
import pandas as pd

df = pd.read_csv('preprocessed_data.csv', index_col=0)

In [4]:
from sklearn.model_selection import train_test_split

functions = df['function'].values
y = df['isVulnerable'].values

functions_train, functions_test, y_train, y_test = train_test_split(functions, y, test_size=0.2, random_state=42)


In [5]:
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import wordpunct_tokenize

tokenizer = Tokenizer(analyzer=wordpunct_tokenize)
tokenizer.fit_on_texts(functions_train)

X_train = tokenizer.texts_to_sequences(functions_train)
X_test = tokenizer.texts_to_sequences(functions_test)

vocab_size = len(tokenizer.word_index) + 1


2023-04-30 12:15:25.901672: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 12:15:25.940668: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 12:15:25.941465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:15:26.565244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
print(vocab_size)

54797


In [7]:
print(functions_train[0])
print(X_train[0])

void rtw_surveydone_event_callback(struct adapter	*adapter, u8 *pbuf)
{
	struct	mlme_priv *pmlmepriv = &(adapter->mlmepriv);

	spin_lock_bh(&pmlmepriv->lock);
	if (pmlmepriv->wps_probe_req_ie) {
		pmlmepriv->wps_probe_req_ie_len = 0;
		kfree(pmlmepriv->wps_probe_req_ie);
		pmlmepriv->wps_probe_req_ie = NULL;
	}

	if (check_fwstate(pmlmepriv, _FW_UNDER_SURVEY)) {
		del_timer_sync(&pmlmepriv->scan_to_timer);
		_clr_fwstate_(pmlmepriv, _FW_UNDER_SURVEY);
	}

	rtw_set_signal_stat_timer(&adapter->recvpriv);

	if (pmlmepriv->to_join) {
		if ((check_fwstate(pmlmepriv, WIFI_ADHOC_STATE) == true)) {
			if (check_fwstate(pmlmepriv, _FW_LINKED) == false) {
				set_fwstate(pmlmepriv, _FW_UNDER_LINKING);

				if (rtw_select_and_join_from_scanned_queue(pmlmepriv) == _SUCCESS) {
					_set_timer(&pmlmepriv->assoc_timer, MAX_JOIN_TIMEOUT);
				} else {
					u8 ret = _SUCCESS;
					struct wlan_bssid_ex    *pdev_network = &(adapter->registrypriv.dev_network);
					u8 *pibss = adapter->registrypriv.dev_ne

In [10]:
from keras.utils import pad_sequences

maxlen = 1000

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0])


[   41 37081     1    10    91    12    91     4   122    12  1966     7
     9    10  7691    12   538     6  1731    91     2  5138     5   382
    14   538     2   101     5     8     1   538     2 17660     7     9
   538     2 37082     6    15     3    86     1   538     2 17660     5
   538     2 17660     6    35     3    11     8     1  2359     1   538
     4 13530    28     9   555    14   538     2 23429     5  4799     1
   538     4 13530     5    11 37083    14    91     2  8529     5     8
     1   538     2 13882     7     9     8   108  2359     1   538     4
 25100     7    38   115    28     9     8     1  2359     1   538     4
  5431     7    38   118     7     9 25101     1   538     4  3686     5
     8     1 13531     1   538     7    38  4094     7     9  6394    14
   538     2  4800     4 13532     5    11    48     9   122    19     6
  4094     3    10 15441    12 25102     6  1731    91     2 25103    13
 25104     5   122    12 25105     6    91     2 25

In [13]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 50)          2739850   
                                                                 
 global_max_pooling1d (Globa  (None, 50)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,740,371
Trainable params: 2,740,371
Non-trainable params: 0
_________________________________________________________________


In [15]:
from keras.backend import clear_session
clear_session()
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))



Epoch 1/20
1960/1960 [==============================] - 66s 34ms/step - loss: 0.6916 - accuracy: 0.5175 - val_loss: 0.6841 - val_accuracy: 0.5324
Epoch 2/20
1960/1960 [==============================] - 65s 33ms/step - loss: 0.6684 - accuracy: 0.5764 - val_loss: 0.6610 - val_accuracy: 0.5834
Epoch 3/20
1960/1960 [==============================] - 66s 34ms/step - loss: 0.6262 - accuracy: 0.6244 - val_loss: 0.6454 - val_accuracy: 0.6105
Epoch 4/20
1960/1960 [==============================] - 62s 32ms/step - loss: 0.5790 - accuracy: 0.6701 - val_loss: 0.6382 - val_accuracy: 0.5971
Epoch 5/20
1960/1960 [==============================] - 63s 32ms/step - loss: 0.5365 - accuracy: 0.7006 - val_loss: 0.6278 - val_accuracy: 0.6226
Epoch 6/20
1960/1960 [==============================] - 58s 29ms/step - loss: 0.5028 - accuracy: 0.7197 - val_loss: 0.6111 - val_accuracy: 0.6326
Epoch 7/20
1960/1960 [==============================] - 58s 30ms/step - loss: 0.4750 - accuracy: 0.7330 - val_loss: 0.5995 -

NameError: name 'plot_history' is not defined